# Traditional Methods trained on every file, tuned on the rest (untuned)
MinMaxScaler is applied to these tests. All models are untuned. The model will be trained on file A, tested on files B,C,D.., then on B, tested on A,C,D.. etc

In [1]:
import sys
sys.path.append("../../")
from time import process_time
from os import listdir, chdir

def warn(*args, **kwargs):
  pass
import warnings
warnings.warn = warn

from modules.NetworkTraffic import NetworkTraffic
from sklearn import model_selection, metrics

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.neural_network import MLPClassifier

FilesToTest = list()
chdir("../../data")
for file in listdir():
  if file.endswith(".csv"):
    FilesToTest.append(file)

TestSize = [0.4]
ModelsToTest = [RandomForestClassifier(), GradientBoostingClassifier(), DecisionTreeClassifier(), MLPClassifier(), LinearSVC()]
OutputResults = dict()
#ModelResults = dict()

In [2]:
def testModel(model, x_train, x_test, y_train, y_test):
  print(f"Testing {str(model)}", end=', ')
  start = process_time()

  ### Begin timing
  temp_clf = model
  temp_clf.fit(x_train, y_train)

  y_pred = temp_clf.predict(x_test)
  ### End timing

  stop = process_time()

  # Results
  tempDict = {
    "Accuracy": metrics.accuracy_score(y_test, y_pred),
    "Balanced Accuracy": metrics.balanced_accuracy_score(y_test, y_pred),
    "F1 Micro": metrics.f1_score(y_test, y_pred, average='micro'),
    "Precision Micro": metrics.f1_score(y_test, y_pred, average='micro'),
    "Recall Micro": metrics.recall_score(y_test, y_pred, average='micro'),
    "Runtime": stop-start,
  }
  return tempDict

In [3]:
def updateModelResults(size, model, results):
  def changeKeyValue():
    try:
      ModelResults[size][model][key] += results[key]
    except KeyError:
      ModelResults[size][model][key] = results[key]
  
  # For each metric, attempt to set or add to the value
  for key in results:
    try:
      changeKeyValue()
    except KeyError:
      ModelResults[size][model] = dict()
      changeKeyValue()

# Divide each metric by the total number of files tested
def findAveragesForModelResults(fileCount):
  for size in ModelResults:
    for model in ModelResults[size]:
      for metric in ModelResults[size][model]:
        ModelResults[size][model][metric] /= fileCount

In [4]:
from copy import deepcopy

OutputResults.clear()
#ModelResults.clear()

for size in TestSize:
  print(f"\nSearching with test size of {size*100}%...")
  OutputResults[size] = dict()
  #ModelResults[size] = dict()

  for index, file in enumerate(FilesToTest):
    print(file, end=', ')
    OutputResults[size][file] = dict()
    currentFileData = NetworkTraffic(file, testSize=size, doNorm=True, doNormAll=True)
    restOfFiles = deepcopy(FilesToTest)
    restOfFiles.pop(index)
    restOfTheFilesData = NetworkTraffic(restOfFiles, testSize=size, doNorm=True, doNormAll=True)
    x_train, x_test, y_train, y_test = currentFileData.data, currentFileData.target, restOfTheFilesData.data, restOfTheFilesData.target

    for model in ModelsToTest:
      #print(f"{file} : {str(model)}...")
      results = testModel(model, x_train, x_test, y_train, y_test)
      OutputResults[size][file].update({str(model): results})
      #updateModelResults(size, str(model), results)

#findAveragesForModelResults(len(FilesToTest))


Searching with test size of 40.0%...
b5000d100.csv, [[2.1940e+03 8.0000e+00 1.0000e+02 ... 1.8000e+00 1.4480e+04 6.4088e+04]
 [1.5380e+03 1.1000e+01 1.0000e+02 ... 2.2000e+00 1.4480e+04 6.4088e+04]
 [1.4180e+03 1.0000e+01 1.0000e+02 ... 1.3000e+00 1.4480e+04 6.4088e+04]
 ...
 [1.3050e+04 0.0000e+00 4.1692e+01 ... 4.7800e+01 1.4480e+04 6.4088e+04]
 [1.2278e+04 1.0000e+00 5.7334e+01 ... 3.8900e+01 1.4480e+04 6.4088e+04]
 [1.3812e+04 0.0000e+00 4.4600e+01 ... 5.5600e+01 1.4480e+04 6.4088e+04]]
[4 4 4 ... 6 6 6]
[[4.9580e+03 2.8000e+01 3.0000e+01 ... 5.0000e+00 1.4480e+04 6.4088e+04]
 [5.7570e+03 2.7000e+01 3.0000e+01 ... 5.3000e+00 1.4480e+04 6.4088e+04]
 [4.5050e+03 3.1000e+01 3.0000e+01 ... 7.3000e+00 1.4480e+04 6.4088e+04]
 ...
 [6.6897e+04 0.0000e+00 1.8973e+01 ... 9.6100e+01 1.4480e+04 6.4088e+04]
 [6.6001e+04 0.0000e+00 1.2819e+01 ... 9.6100e+01 1.4480e+04 6.4088e+04]
 [6.5989e+04 0.0000e+00 1.6517e+01 ... 9.6100e+01 1.4480e+04 6.4088e+04]]
[4 4 4 ... 6 6 6]
Testing RandomForestCla

ValueError: Unknown label type: 'continuous-multioutput'

In [ ]:
import json
with open("EveryFileTransfer_Untuned_AllTestResults.json", "w") as f:
  f.write(json.dumps(OutputResults, indent=2))

In [ ]:
with open("EveryFileTransfer_Untuned_ModelResults.csv", "w") as f3:
  f3.write("File Trained On,Model,Accuracy,Runtime\n")
  for size in OutputResults:
    for file in OutputResults[file]:
      for model in OutputResults[size][file]:
        f3.write(f"{file},{model},{OutputResults[size][file][model]['Accuracy']},{OutputResults[size][file][model]['Runtime']}\n")